In [7]:
import pandas as pd
import requests


codnas=pd.read_csv("../data/download_codnas_biolip_full.tsv", sep="\t", engine='python')
codnas


In [13]:
#create a single list of unique pdb ids grouped from PDB_1 and PDB_2 columns based on the same value in Pool_repr column
pdb_ids = codnas.groupby('Pool_repr').apply(lambda x: list(set(x['PDB_1'].tolist() + x['PDB_2'].tolist()))).reset_index()

#add column that removes the _* from the Pool_repr column
pdb_ids['PDB_ID'] = pdb_ids['Pool_repr'].apply(lambda x: x.split('_')[0])

pdb_ids

/var/folders/98/hv8x634d1fg2b1_0kpth665h0000gn/T/ipykernel_19683/1411366118.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pdb_ids = codnas.groupby('Pool_repr').apply(lambda x: list(set(x['PDB_1'].tolist() + x['PDB_2'].tolist()))).reset_index()


,Pool_repr,0,PDB_ID
0,1AAT_A,"[2CST_A, 1AAT_B, 2CST_B, 1AAT_A]",1AAT
1,1AUO_A,"[1AUR_A, 1AUO_A, 1AUR_B, 1AUO_B]",1AUO
2,1BCP_D,"[1BCP_E, 1PTO_K, 1PTO_E, 1PRT_D, 1BCP_D, 1BCP_...",1BCP
3,1BH8_B,"[1BH9_B, 1BH8_B]",1BH8
4,1BZF_A,"[1LUD-11_A, 2HQP-5_A, 1AO8-18_A, 2HM9-4_A, 1DI...",1BZF
...,...,...,...
208,6V4T_A,"[6E8W-4_A, 6E8W-15_B, 6V4T-3_A, 6E8W-8_B, 6V4T...",6V4T
209,6W70_A,"[6W70_C, 6W6X_B, 6X8N_B, 6W6X_A, 6W70_A, 6X8N_A]",6W70
210,6XTG_L,"[6XUD_L, 6XTG_L]",6XTG
211,6YAN_f,"[6YAL_f, 6ZVH_f, 6YAN_f]",6YAN


In [15]:
def fetch_pdb_data(pdb_id):
    url = "https://data.rcsb.org/graphql"

    query = """
    {
      entries(entry_ids:["%s"]){
        polymer_entities {
          rcsb_id
          rcsb_polymer_entity_container_identifiers {
            reference_sequence_identifiers {
              database_accession
              database_name
            }
          }
        }
      }
    }
    """ % pdb_id

    response = requests.post(url, json={'query': query})

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Query failed with status code: {response.status_code}")
        print(response.text)
        return None

#write def to get the database_accession from result
def get_uniprot_accession(pdb_id):
    result = fetch_pdb_data(pdb_id)
    if result:
        try:
            db_accession = result['data']['entries'][0]['polymer_entities'][0]['rcsb_polymer_entity_container_identifiers']['reference_sequence_identifiers'][0]['database_accession']
            return db_accession
        except Exception as e:
            print(f"Error: {e}")
            return None
    else:
        return None


# Example usage:
pdb_id = "1AAT"
result = get_uniprot_accession(pdb_id)
if result:
    print(result)

P00504


In [16]:

def fetch_uniprot_sequence(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # Split the response text into lines
        lines = response.text.split('\n')
        
        # The first line is the header, we'll skip it
        sequence = ''.join(lines[1:])
        
        return sequence.strip()
    else:
        print(f"Failed to fetch sequence. Status code: {response.status_code}")
        print(response.text)
        return None

# Example usage
sequence = fetch_uniprot_sequence("P00504")
sequence

'MAASIFAAVPRAPPVAVFKLTADFREDGDSRKVNLGVGAYRTDEGQPWVLPVVRKVEQLIAGDGSLNHEYLPILGLPEFRANASRIALGDDSPAIAQKRVGSVQGLGGTGALRIGAEFLRRWYNGNNNTATPVYVSSPTWENHNSVFMDAGFKDIRTYRYWDAAKRGLDLQGLLDDMEKAPEFSIFILHACAHNPTGTDPTPDEWKQIAAVMKRRCLFPFFDSAYQGFASGSLDKDAWAVRYFVSEGFELFCAQSFSKNFGLYNERVGNLSVVGKDEDNVQRVLSQMEKIVRTTWSNPPSQGARIVATTLTSPQLFAEWKDNVKTMADRVLLMRSELRSRLESLGTPGTWNHITDQIGMFSFTGLNPKQVEYMIKEKHIYLMASGRINMCGLTTKNLDYVAKSIHEAVTKIQ'

In [ ]:
#add column to pdb_ids with uniprot accession using the get_uniprot_accession function
pdb_ids['uniprot_accession'] = pdb_ids['PDB_ID'].apply(get_uniprot_accession)

#add column to pdb_ids with uniprot accession using the get_uniprot_accession function
pdb_ids['sequence'] = pdb_ids['uniprot_accession'].apply(fetch_uniprot_sequence)



In [21]:
#export to csv
pdb_ids.to_csv("../data/codnas_dataset.csv", index=False)